In [ ]:
# PS6 - CE264
# GSI: Mustapha Harb, Mengqiao Yu, Andrew Campbell

# importing the requried libraries
from collections import OrderedDict    # For recording the model specification 

import pandas as pd                    # For file input/output
import numpy as np                     # For vectorized math operations

import pylogit as pl                   # For MNL model estimation and
                                       # conversion from wide to long format
    
%matplotlib inline

# reading the data file 
data_wide  = pd.read_csv("data01.csv",sep=",")

# Problem 1: MNL Evaluation

## Build the MNL specifications to predict mode choice 

In [ ]:
# converting the data from wide to long format

# Create the list of individual specific variables
ind_variables = data_wide.columns.tolist()[:2] + ["weights"]

# Specify the variables that vary across individuals and some or all alternatives
# The keys are the column names that will be used in the long format dataframe.
# The values are dictionaries whose key-value pairs are the alternative id and
# the column name of the corresponding column that encodes that variable for
# the given alternative. Examples below.
alt_varying_variables = {u'travel_time': dict([(1, 'tt_da'),
                                               (2, 'tt_sr'),
                                               (3, 'tt_walk'),
                                               (4, 'tt_bike'),
                                               (5, 'tt_wt'),
                                               (6, 'tt_dt')]),
                          u'distance_car': dict([(1, 'dist_car'),
                                                (2, 'dist_car')]),
                          u'travel_cost': dict([(1, 'cost_da'),
                                                (2, 'cost_sr'),
                                                (5, 'cost_wt'),
                                                (6, 'cost_dt')]),
                          u'access_time': dict([(5, 'accTime_wt'),
                                                (6, 'accTime_dt')]),
                          u'egress_time': dict([(5, 'egrTime_wt'),
                                                (6, 'egrTime_dt')]),
                          u'initial_wait': dict([(5, 'iWait_wt'),
                                                 (6, 'iWait_dt')]),
                          u'transfer_wait': dict([(5, 'xWait_wt'),
                                                  (6, 'xWait_dt')]),
                          u'access_distance_dt': dict([(6, "accDist_dt")])}

# Specify the availability variables
# Note that the keys of the dictionary are the alternative id's.
# The values are the columns denoting the availability for the
# given mode in the dataset.


availability_variables = {1: 'avail_da',
                          2: 'avail_sr', 
                          3: 'avail_walk',
                          4: 'avail_bike',
                          5: 'avail_wt',
                          6: 'avail_dt'}

##########
# Determine the columns for: alternative ids, the observation ids and the choice
##########
# The 'custom_alt_id' is the name of a column to be created in the long-format data
# It will identify the alternative associated with each row.
custom_alt_id = "mode_id"

# Create a custom id column that ignores the fact that this is a 
# panel/repeated-observations dataset. Note the +1 ensures the id's start at one.
obs_id_column = "obsID"

# Create a variable recording the choice column
choice_column = "choice"

In [ ]:
# Perform the conversion to long-format
data_long = pl.convert_wide_to_long(data_wide, 
                                           ind_variables, 
                                           alt_varying_variables, 
                                           availability_variables, 
                                           obs_id_column, 
                                           choice_column,
                                           new_alt_id_name=custom_alt_id)

In [ ]:
##########
# Create scaled variables so the estimated coefficients are of similar magnitudes
##########
# Scale the travel time column by 60 to convert raw units (minutes) to hours
data_long["travel_time_hrs"] = data_long["travel_time"] / 60.0

# Scale the access by 60 to convert raw units (minutes) to hours
data_long["access_time_hrs"] = data_long["access_time"] / 60.0

# for drive to transit let us combine travel time and access time
data_long["travel_time_access_time_hrs"] = data_long["travel_time_hrs"] + data_long["access_time_hrs"]

#Scale the egress time by 60
data_long["egress_time_hrs"] = data_long["egress_time"] / 60.0

# combining access and egress time which we want to use for the walk to transit alternative
data_long["acess_egress_hrs"] = data_long["access_time_hrs"] + data_long["egress_time_hrs"]

# scaling the initial wait by 60
data_long["initial_wait_hrs"] = data_long["initial_wait"] / 60.0

# scaling the transfer wait by 60
data_long["transfer_wait_hrs"]  = data_long["transfer_wait"] / 60.0

# combining transfer wait and initial wait to be used for walk to transit and bike to transit
data_long["initial_transfer_wait_hrs"] = data_long["initial_wait_hrs"] + data_long["transfer_wait_hrs"]


# creating non-linear transformations for the cost variable
cutOff1 = 2
cutOff2 = 7


data_long["cost_cat_one"] = (data_long["travel_cost"] <= cutOff1)*data_long["travel_cost"] + (data_long["travel_cost"] > cutOff1)*cutOff1

data_long["cost_cat_two"] = (data_long["travel_cost"] > cutOff1)*(data_long["travel_cost"] <= cutOff2)*(data_long["travel_cost"] - cutOff1) + (data_long["travel_cost"] > cutOff2)* (cutOff2 - cutOff1)

data_long["cost_cat_three"] = (data_long["travel_cost"] > cutOff2)*(data_long["travel_cost"] - cutOff2)



# specifying the utility equations

# NOTE: - Specification and variable names must be ordered dictionaries.
#       - Keys should be variables within the long format dataframe.
#         The sole exception to this is the "intercept" key.
#       - For the specification dictionary, the values should be lists
#         of integers or or lists of lists of integers. Within a list, 
#         or within the inner-most list, the integers should be the 
#         alternative ID's of the alternative whose utility specification 
#         the explanatory variable is entering. Lists of lists denote 
#         alternatives that will share a common coefficient for the variable
#         in question.

basic_specification = OrderedDict()
basic_names = OrderedDict()


basic_specification["intercept"] = [ 2, 3, 4, 5, 6]
basic_names["intercept"] = ['ASC SR',
                            'ASC Walk', 'ASC Bike', 'ASC WT', 'ASC DT']

basic_specification["travel_time_hrs"] = [[1, 2, 5], 4, 3]
basic_names["travel_time_hrs"] = ['In-Vehicle Travel Time, units:hrs (DA, SR, WT)',
                                  'Bike Time, units:hrs (Bike)',
                                  'Walk Time, units:hrs (Walk)']

basic_specification["travel_time_access_time_hrs"] = [6]
basic_names["travel_time_access_time_hrs"] = ["In-Vehicle Travel Time, units:hrs, (DT)"]

basic_specification["acess_egress_hrs"] = [5]
basic_names["acess_egress_hrs"] = ["Walk Time, units:hrs, (WT)"]

basic_specification["egress_time_hrs"] = [6]
basic_names["egress_time_hrs"] = ["Walk Time, units:hrs, (DT)"]

basic_specification["initial_transfer_wait_hrs"] = [[5, 6]]
basic_names["initial_transfer_wait_hrs"] = ["Waiting Time, units:hrs, (WT and DT)"]


basic_specification["cost_cat_one"] = [[1, 2, 5,6]]
basic_names["cost_cat_one"] = ['Cost: Under $2']

basic_specification["cost_cat_two"] = [[1, 2, 5,6]]
basic_names["cost_cat_two"] = ['Cost: (2 - 7)$']

basic_specification["cost_cat_three"] = [[1, 2, 5,6]]
basic_names["cost_cat_three"] = ['Cost: Above $7']



## 1.a - Split into training and test data

In [ ]:
### taking a sample of 10,000 observation from the BATS 2000 dataset 
# and split it into train and test sets
train_size = 8000
total_size = 10000

# Suffle the data before sampling
np.random.seed(1138)  
shuffled = np.random.choice(data_long.obsID.unique()[0:total_size], total_size, replace=False)
train_ids = shuffled[0:train_size]
test_ids = shuffled[train_size:]

new_data_train = data_long.loc[data_long[obs_id_column].isin(train_ids)].copy()
new_data_test = data_long.loc[data_long[obs_id_column].isin(test_ids)].copy()

In [ ]:
print train_ids.size
print test_ids.size

### Estimate model and inspect results

In [ ]:
# Estimate the multinomial logit model (MNL)
data_mnl = pl.create_choice_model(data=new_data_train,
                                  alt_id_col=custom_alt_id,
                                  obs_id_col=obs_id_column,
                                  choice_col=choice_column,
                                  specification=basic_specification,
                                  model_type="MNL",
                                  names=basic_names)

# Specify the initial values and method for the optimization.
data_mnl.fit_mle(np.zeros(15))

In [ ]:
# Look at the estimation results
data_mnl.get_statsmodels_summary()

## 1.b - MNL Prediction

#### Use the model estimated on the *training data* to predict the probability of each mode for the *test data* 

In [ ]:
prediction_array_test = data_mnl.predict(new_data_test)

In [ ]:
#add the predicted probabilities to the test dataset
new_data_test['predict'] = prediction_array_test

### Calculate the accuracy of our MNL model for the *test data*

In [ ]:
idx_test = new_data_test.groupby(u'obsID').apply(lambda x: np.argmax(x.predict))  # indices of predicted choices in the long data
predicted_actual_test = new_data_test.loc[idx_test, :]['choice']  # actual outcomes for the predicted choices                                                                                                              # [0 = pred wrong, 1 = pred right]
accuracy_test = np.true_divide(np.sum(predicted_actual_test), predicted_actual_test.size)
print accuracy_test

<font color=red> 
### The accuracy of the MNL model is 0.696

## Evaluate the model across a range of training sizes

### Training and test accuracy

Make a plot comparing training and test accuracy for differenet training sample size. Keep the total_size fixed at 10,000, but range across the train_size. Make sure you clude some small training sample sizes. I recommend using some different intervals like: [100, 200, 500, 1000, 2000, 3000, ... 9000]

Use the above code as template. We recommend using a for loop to iterate through all the train_size values instead of doing it manually.

### Helpful functions

The following two functions should simplify things for your for loops.

In [ ]:
# Function to return shuffled training and data sets
def split_data(data, train_size, test_size, seed=1138):
    """
    Takes a long-form DataFrame and returns mutually exclusive training and test samples of specificed lengths. Input
    data observation ids are shuffled to ensure random draws for samples.
    
    Keyword arguments:
    data -- long format DataFrame
    train_size -- integer size of training sample
    test_size -- integer size of test sample
    seed -- integer to set random seed for Numpy
    """
    # Suffle the data before sampling
    np.random.seed(seed)  
    shuffled = np.random.choice(data.obsID.unique()[0:train_size+test_size], train_size+test_size, replace=False)
    train_ids = shuffled[0:train_size]
    test_ids = shuffled[train_size:]

    train = data.loc[data[obs_id_column].isin(train_ids)].copy()
    test = data.loc[data[obs_id_column].isin(test_ids)].copy()
    return (train, test)

In [ ]:
# Function to calculate the accuracy and log-likelihood using a trained MNL and data
def eval_prediction(model, data):
    """
    Function to calculate to evaluate model estimates. It returns a tuple of 1) accuracy, 2) log-likelihood
    3) null log-likelihood  4) likelihood ratio index and 5) adjusted likelihood ratio index
    
    Keyword arguments:
    model -- a trained PyLogit model
    data -- (N x K) Pandas DataFrame long format. Same attributes as that used to train the model
    
    returns -- Tuple: (accuracy, log-likelihood, null log-likelihood, likelihood ratio index,
    adjusted likelihood ratio index)
    """
    ##
    # Calculate the accuracy
    ##
    
    # Predicted choice probabilities 
    prediction_array = model.predict(data)
    data['predict'] = prediction_array
    idx = data.groupby(u'obsID').apply(lambda x: np.argmax(x.predict))  # indices of predicted choices in the long data
    predicted_actual = data.loc[idx, :]['choice']  # actual outcomes for the predicted choices
    accuracy = np.true_divide(np.sum(predicted_actual), predicted_actual.size)
    
    ##
    # Calculate the log-likelihood
    ##
    ll = np.dot(data.choice, np.log(prediction_array))
    
    ##
    # Calculate the likelihood ratio index
    ##

    # calc the null predictions
    null_prediction_array = data.groupby(u'obsID')[u'obsID'].transform(lambda x: np.true_divide(1, x.shape[0]))
    ll_0 = np.dot(data.choice, np.log(null_prediction_array))
    rho_2 = 1 - np.abs(np.true_divide(ll,ll_0))
    
    ##
    # Calc adjusted likelihood ratio index
    ##
    rho_bar_2 = 1 - np.abs(np.true_divide(ll - model.params.shape[0], ll_0))
    
    return (accuracy, ll, ll_0, rho_2, rho_bar_2)

### Log-likelihood

Plot the training and test log-likelihoods of the model for the same training sizes used above. Either add a second y-axis to the accuracy plot, or make a new plot.

=======================================================================================================================

# Problem 2: Use ML algorithms we learned in class to predict mode choice

<font color=blue>
### The next 3 cells are just to fix the data to match the dataset used to train the MNL model

In [ ]:
df = data_wide
df = df[:9999]
np.random.seed(100)

In [ ]:
mins_to_hours_cols = ['tt_da', 'tt_sr', 'tt_walk', 'tt_bike', 
                     'tt_wt', 'accTime_wt', 'egrTime_wt',
                     'iWait_wt', 'xWait_wt', 'tt_dt', 
                     'accTime_dt','egrTime_dt', 'iWait_dt', 
                     'xWait_dt']

for col in mins_to_hours_cols:
    df[col+"_hrs"] = df[col]/60

df["travel_time_access_time_hrs_dt"] = df["tt_dt_hrs"] + df["accTime_dt_hrs"]
df["travel_time_access_time_hrs_wt"] = df["tt_wt_hrs"] + df["accTime_wt_hrs"]

df["access_egress_hrs_dt"] = df["accTime_dt_hrs"] + df["egrTime_dt_hrs"]
df["access_egress_hrs_wt"] = df["accTime_wt_hrs"] + df["egrTime_wt_hrs"]

df["initial_transfer_wait_hrs_dt"] = df["iWait_dt_hrs"] + df["xWait_dt_hrs"]
df["initial_transfer_wait_hrs_wt"] = df["iWait_wt_hrs"] + df["xWait_wt_hrs"]

# creating non-linear transformations for the cost variable
cutOff1 = 2
cutOff2 = 7

df["cost_cat_one_da"] = (df['cost_da']<=cutOff1)*df['cost_da'] + (df['cost_da'] > cutOff1)*cutOff1
df["cost_cat_two_da"] = (df['cost_da'] > cutOff1)*(df['cost_da'] <= cutOff2)*(df['cost_da'] - cutOff1) + (df['cost_da'] > cutOff2)* (cutOff2 - cutOff1)
df["cost_cat_three_da"] = (df['cost_da'] > cutOff2)*(df['cost_da'] - cutOff2)

df["cost_cat_one_sr"] = (df['cost_sr']<=cutOff1)*df['cost_sr'] + (df['cost_sr'] > cutOff1)*cutOff1
df["cost_cat_two_sr"] = (df['cost_sr'] > cutOff1)*(df['cost_sr'] <= cutOff2)*(df['cost_sr'] - cutOff1) + (df['cost_sr'] > cutOff2)* (cutOff2 - cutOff1)
df["cost_cat_three_sr"] = (df['cost_sr'] > cutOff2)*(df['cost_sr'] - cutOff2)

df["cost_cat_one_dt"] = (df['cost_dt']<=cutOff1)*df['cost_dt'] + (df['cost_dt'] > cutOff1)*cutOff1
df["cost_cat_two_dt"] = (df['cost_dt'] > cutOff1)*(df['cost_dt'] <= cutOff2)*(df['cost_dt'] - cutOff1) + (df['cost_dt'] > cutOff2)* (cutOff2 - cutOff1)
df["cost_cat_three_dt"] = (df['cost_dt'] > cutOff2)*(df['cost_dt'] - cutOff2)

df["cost_cat_one_wt"] = (df['cost_wt']<=cutOff1)*df['cost_wt'] + (df['cost_wt'] > cutOff1)*cutOff1
df["cost_cat_two_wt"] = (df['cost_wt'] > cutOff1)*(df['cost_wt'] <= cutOff2)*(df['cost_wt'] - cutOff1) + (df['cost_wt'] > cutOff2)* (cutOff2 - cutOff1)
df["cost_cat_three_wt"] = (df['cost_wt'] > cutOff2)*(df['cost_wt'] - cutOff2)


In [ ]:
all_cols = pd.DataFrame(df.columns,columns=["col_name"])

In [ ]:
cols = list(df.columns.values)
cols_to_include = (cols[45:] + ["egrTime_wt_hrs","egrTime_dt_hrs"]+
                   cols[23:29] + cols[31:36] + ["tt_dt_hrs"] + ["choice"])

new_df = df[cols_to_include]

### use sklearn to split the data into a training and testing sets

In [ ]:
from sklearn import model_selection

X = new_df.drop(["choice"],axis=1)
y = new_df["choice"]
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, 
                                                                    test_size=0.2, 
                                                                    shuffle = False)

## Example code:Training and testing a KNN algorithm

In [ ]:
# step 1: import the sklarn package that allows you to do KNN in python
from sklearn import neighbors

In [ ]:
# step 2: set up the name of your classifier and the hyper-parameters you want to use
KNN_classifier = neighbors.KNeighborsClassifier(n_neighbors=5)

In [ ]:
# step 3: train/fit the classifier on training data
KNN_classifier.fit(X=X_train,y=y_train)

In [ ]:
# step 4: use your clasifier to predict mode choice for test data
KNN_predictions = KNN_classifier.predict(X_test)

In [ ]:
# step 5: compare your predictions with the actual choices
KNN_predictions == y_test

In [ ]:
# step 6: calculate accuracy (also called score)
sum(KNN_predictions == y_test)/float(len(KNN_predictions))

step 7: repeat all steps for a different hyper parameter (in this case number of neighbers K). 
<font color=red> 
*best way to do so is run a for loop instead of repeating the steps and changing the parameter by hand*

<font color=red> Note: an easier way to do steps 4, 5 and 6 is to use the *score* function in sklearn (see example below).

I highly recommend you do not do that for this assignment to learn more by doing things manually rather than running a line of code that does things for you without really understanding much

In [ ]:
# Example:
KNN_classifier.score(X = X_test, y = y_test)

### plot graph of accuracy vs. number of neighors for train dataset

### plot graph of accuracy vs. number of neighors for test dataset

# Train and test a decision tree classifier

In [ ]:
from sklearn import tree

### plot graph of accuracy vs. hyper_parameters for train dataset

### plot graph of accuracy vs. hyper_parameters for test dataset

# Train and test a Random forest classifier

### plot graph of accuracy vs. hyper_parameters for train dataset

### plot graph of accuracy vs. hyper_parameters for test dataset

# Train and test a SVM classifier

### plot graph of accuracy vs. hyper_parameters for train dataset

### plot graph of accuracy vs. hyper_parameters for test dataset

# Create table for all the algorithms you used and their highest accuracy (including your MNL model)